# Calculate Simplified Ikeda and insert into DB

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

from rolldecay.simplified_ikeda import calculate_roll_damping
from rolldecay import equations
import sympy as sp
from rolldecay import symbols
from rolldecayestimators.substitute_dynamic_symbols import lambdify

import data
import copy
from rolldecay import database
from mdldb.tables import RolldecaySimplifiedIkeda

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_linear')

In [ ]:
def calculate(row):
    
    LPP = row.lpp
    Beam = row.beam
    DRAFT = (row.TA + row.TF)/2
    
    PHI = row.phi_max
    lBK = row.BKL
    bBK = row.BKB
    OMEGA = row.omega0
    OG = (-row.kg+DRAFT)
    CB = row.Volume/(row.lpp*row.beam*DRAFT)
    CMID = row.A0
    
    B44HAT, BFHAT, BWHAT, BEHAT, BBKHAT = calculate_roll_damping(LPP,Beam,CB,CMID,OG,PHI,lBK,bBK,OMEGA,DRAFT)
    s = pd.Series()
    s['B44HAT'] = B44HAT
    s['BFHAT'] = BFHAT
    s['BWHAT'] = BWHAT
    s['BEHAT'] = BEHAT
    s['BBKHAT'] = BBKHAT
    s['DRAFT'] = DRAFT
    s['PHI'] = PHI
    s['bBK'] = bBK
    s['lBK'] = lBK
    s['OMEGA'] = OMEGA
    s['OG'] = OG
    s['CB'] = CB
    s['CMID'] = CMID
    
    return s
    

In [ ]:
required = ['lpp','beam','TA','TF','BKL','BKB','omega0','kg','Volume','A0']

In [ ]:
df_rolldecay.dropna(subset=required, inplace=True)

In [ ]:
mask = ((df_rolldecay['ship_speed']==0) &  ## Zero speed!
        (df_rolldecay[required] > 0).all(axis=1)
       )
df = df_rolldecay.loc[mask].copy()
df['phi_max'] = np.rad2deg(df['phi_start'].abs())

In [ ]:
result = df.apply(func=calculate, axis=1)
df = pd.merge(left=df,right=result,left_index=True, right_index=True)

In [ ]:
result.head()

In [ ]:
db = database.get_db()

In [ ]:
for run_id, row in result.iterrows():
    
    run = RolldecaySimplifiedIkeda(run_id=run_id,**row)
    db.session.merge(run)
db.session.commit()